# Treinar o Modelo de Classificação
---
Carregar os Dados
Definir a Variável Alvo e Preditoras

Cross-Validation: Utilizar uma abordagem de validação cruzada (k-fold cross-validation) para avaliar o desempenho do modelo em múltiplas partições, o que é particularmente útil para um dataset pequeno.

Treinar o Modelo: Utilizar um modelo de classificação, como H2O Random Forest ou H2O Gradient Boosting Machine (GBM), configurado com cross-validation.

Analisar a Variância Entre Modelos Intermédios: Avalie os modelos gerados pela cross-validation para verificar se existe uma variância significativa entre eles. Isso pode indicar a estabilidade do modelo.

Avaliar Métricas de Performance: Examine métricas como AUC (Área Sob a Curva), Log Loss, Precision e Recall para determinar a capacidade preditiva do modelo.

In [9]:
import h2o
from h2o.estimators import H2ORandomForestEstimator
import numpy as np

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,2 hours 35 mins
H2O_cluster_timezone:,Europe/Lisbon
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,10 days
H2O_cluster_name:,H2O_from_python_avlal_rc0d8l
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.885 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [10]:

# Carregar o dataset
data_heart = h2o.import_file("..\\data\\heart.csv")

# Definir a variável alvo e preditoras
target = "output"
predictors = [col for col in data_heart.columns if col != target]




Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [11]:
# Converter a variável alvo para categórica
data_heart[target] = data_heart[target].asfactor()

# Configurar e treinar o modelo de classificação com cross-validation
model = H2ORandomForestEstimator(nfolds=5, seed=42)
model.train(x=predictors, y=target, training_frame=data_heart)

# Analisar o desempenho do modelo
performance = model.model_performance()
print("Métricas de Performance do Modelo:")
print(performance)

# Obter métricas intermediárias de cross-validation
print("\nResultados de Cross-Validation:")
print("AUC Médio: ", model.auc(xval=True))
print("Log Loss Médio: ", model.logloss(xval=True))

# Avaliar variabilidade entre os modelos de cross-validation
cross_val_models = model.cross_validation_models()
auc_values = [m.auc() for m in cross_val_models]
auc_variance = np.var(auc_values)
print("Variância entre AUC dos modelos de cross-validation:", auc_variance)


drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Métricas de Performance do Modelo:
ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.1330737128637332
RMSE: 0.36479269847919543
LogLoss: 0.5158008494379946
Mean Per-Class Error: 0.18432147562582346
AUC: 0.8892621870882741
AUCPR: 0.8975560075438885
Gini: 0.7785243741765482

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.375
       0    1    Error    Rate
-----  ---  ---  -------  ------------
0      98   40   0.2899   (40.0/138.0)
1      13   152  0.0788   (13.0/165.0)
Total  111  192  0.1749   (53.0/303.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.375        0.851541  89
max f2                       0.2          0.89628   112
max f0point5                 0.708333     0.842105  47
max accuracy              

## Interpretação dos Resultados

#Análise de Variância Entre Modelos: 
 - Se a variância dos valores AUC entre os modelos de cross-validation for alta, o modelo pode ser instável, o que pode reduzir sua confiabilidade para produção.

#Qualidade do Modelo para Produção: 
- Com base nas métricas de AUC, Log Loss, Precision e Recall, avalie se o modelo tem precisão e estabilidade suficientes para uso em produção.

---

#Conclusão

Se a Variância For Baixa e o AUC Elevado:
- O modelo é estável e com boa capacidade preditiva, sendo adequado para produção.
Se a Variância For Alta:
- Um ajuste adicional nos hiperparâmetros ou o uso de técnicas como ensemble pode melhorar  estabilidade do modelo antes de sua utilização em produção.